In [115]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [116]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1825   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)

distrTypeX = LogNormal{dtype}
X_mu = dtype.(round.(rand(B), digits=3))
X_sigma = dtype.(round.(rand(B) * 0.6, digits=3))
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Exponential{dtype}
Y_inv_scale = dtype.(round.(rand(B), digits = 3))
distrY = map(distrTypeY, Y_inv_scale)

@show distrX[1:2]
@show distrY[1:2]

deltas = @. mean(distrX) - mean(distrY);

distrX[1:2] = LogNormal{Float32}[LogNormal{Float32}(μ=0.906f0, σ=0.572f0), LogNormal{Float32}(μ=0.443f0, σ=0.508f0)]
distrY[1:2] = Exponential{Float32}[Exponential{Float32}(θ=0.965f0), Exponential{Float32}(θ=0.071f0)]


In [117]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


8×1800 Matrix{Float32}:
 0.133637    0.0162722   0.0698903  0.0643012   …  0.0285788    0.12464
 0.120016    0.0285583   0.0353553  0.0638659      0.109599     0.00707607
 0.00606461  0.136366    0.0799037  0.128313       0.0400807    0.0653869
 0.0377555   0.0749071   0.0802353  0.177147       0.0514611    0.117508
 0.0178158   0.0233403   0.119965   0.0830359      0.144351     0.0954628
 0.0790322   0.152405    0.191122   0.0151772   …  0.000609229  0.0924027
 0.0465132   0.032767    0.100552   0.129687       0.00699415   0.0157552
 0.107458    0.00534581  0.0410884  0.00296156     0.0154778    0.198447

In [118]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

8×1800 Matrix{Float32}:
 0.133637    0.0162722   0.0698903  0.0643012   …  0.0285788    0.12464
 0.120016    0.0285583   0.0353553  0.0638659      0.109599     0.00707607
 0.00606461  0.136366    0.0799037  0.128313       0.0400807    0.0653869
 0.0377555   0.0749071   0.0802353  0.177147       0.0514611    0.117508
 0.0178158   0.0233403   0.119965   0.0830359      0.144351     0.0954628
 0.0790322   0.152405    0.191122   0.0151772   …  0.000609229  0.0924027
 0.0465132   0.032767    0.100552   0.129687       0.00699415   0.0157552
 0.107458    0.00534581  0.0410884  0.00296156     0.0154778    0.198447

In [119]:
function save_permutation_results(results, B, S, dir="./")
    i = 1
    per_method = []
    for is_two_sided in [true, false]
        for pooled in [true, false]
            averages = []

            for row in eachrow(results[i,:,:])
                coverage = sum([j for (j, _) in row]) / S
                width = sum([j for (_, j) in row]) / S
                push!(averages, (coverage, width))
            end

            alpha_ = is_two_sided ? alpha : alpha / 2
            save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, is_two_sided, parent_dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 2 methods)

In [120]:
B = 100
S = 1825

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]

    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha, twoSided, twoSided)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller, 0.0005)
    
    """
    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end

save_permutation_results(results, B, S)

2464.410118 seconds (1.35 G allocations: 14.896 TiB, 70.52% gc time, 0.02% compilation time)


---

In [28]:
averages = []
for row in eachrow(results[1,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.95, 3.1827745949945476)
 (0.92, 1.6322506846556246)
 (0.85, 1.3739451044574138)
 (0.93, 0.21217739206443034)
 (0.9, 0.18690359039665994)

In [70]:
x = X[:,1,1]
y = Y[:,1,1]
@time permInterval(x, y, deltas[1], (px, py, 0), true, alpha, twoSided, twoSided)

permInterval(x, y, deltas[1], (px, py, 0), true, alpha/2, greater, smaller)

pval(x, y, 2.4218399175319254, (px, py, 0), true, twoSided)

  0.292847 seconds (510.31 k allocations: 55.944 MiB, 93.62% compilation time)


0.05034965f0